In [ ]:
import requests
import json
from getpass import getpass

In [ ]:
# using authentication [user,pass] to get a token from the historian
def getSenderUserToken(auth,url):
    method = 'getUserToken'
    payload = "{\r\n  \"username\":\"%s\",\r\n  \"password\":\"%s\"\r\n}"%(auth[0],auth[1])
    headers = {'Content-Type': 'application/json','Content-Type': 'application/json'}
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    userToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        return userToken
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

# using the getSenderUserToken and a post to the getSessionToken endpoint to return both tokens
def getSenderTokens(url):
    method = 'getSessionToken'
    # requesting user and pass
    print('\nEnter your username: ')
    user = input()
    print('\nEnter your password: ')
    pswd = getpass()
    auth = [user,pswd]
    userToken = getSenderUserToken(auth,url)
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"historians\":[\"hostname\"],\r\n  \"clientId\":\"pythonWriteAPI\",\r\n  \"settings\":{\r\n  \t\"clientTimeout\":300000,\r\n  \t\"fileSize\":8,\r\n  \t\"autoCreateDatasets\": true,\r\n  \t\"autoWriteNoData\": false\r\n  }\r\n}"%userToken
    headers = {
      'Content-Type': 'application/json',
      'Content-Type': 'text/plain'
    }
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    sessToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        return (userToken,sessToken)
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))
        return ('error','error')

# function for making the payload for a storeData post of a single TVQ
def storeDataMakePayload(tag,timestamp,value,utoken,stoken):
    payload = "%s\": [\r\n  \t\t[\r\n  \t\t\"%s\",\r\n  \t\t%s\r\n  \t\t]\r\n  \t]\r\n  }\r\n}"%(tag,timestamp,value)
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"sessionToken\":\"%s\",\r\n  \"tvqs\": {\r\n  \t\""%(utoken,stoken) + payload
    return payload

# function for reading in a CSV and recursively writing each TVQ to the historian
# NOTE: this function assumes the file name is the tag you want to write to
# NOTE: this function is only for storing a single stream of data,
#       where the 1st column is the TIMESTAMP and 2nd column is the VALUE
def storeDataSingleStream(url):
    method = 'storeData'
    data = []
    headers = {
      'Content-Type': 'application/json',
      'Content-Type': 'text/plain'
    }
    utoken,stoken = getSenderTokens(url)
    print('\nEnter the full path to the file, including file name extension (i.e. include .csv or .txt): ')
    filepath = input()
    print('\nEnter the dataset to write to: ')
    name = '.'.join(filepath.split('\\')[-1].split('.')[:-1])
    dataset = input()
    print('\nEnter a branch you want to write to, or leave blank to use the filename as the branch: ')
    branch = input()
    if branch == '':
        tag = dataset + '.' + filepath.split('\\')[-1].split('.')[0]
    else:
        tag = dataset + '.' + branch + '.' + name
    with open(filepath) as f:
        raw = f.readlines()
        for r in raw:
            data += [r.replace('\n','').split(',')]
    data = data[1:]
    for d in data:
        payload = storeDataMakePayload(tag,d[0],d[1],utoken,stoken)
        response = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
        parsed = json.loads(response)
        if parsed.get('statusCode') == 'Good':
            print(response)
        else:
            print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

In [ ]:
url = "https://hostname:port/api/v1/"

In [ ]:
storeDataSingleStream(url)